In [1]:
from datetime import datetime, timedelta
import pandas as pd

Daten laden und vorbereiten

In [2]:
# Lade die CSV-Dateien mit Personen, Teilaufgaben und Projekten
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")

In [3]:
# Wandle die Kompetenzen in eine Liste um, um Filterung zu erleichtern
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(r",\s*")

Initialisierung

In [4]:
# Belegungstracker: Speichert die bereits verplante Kapazität je Person und Monat
belegungen = {}

In [5]:

# Neue Hauptlogik – stundenbasierte Verplanung ohne projektbasierte Blockierung

# Vorbereitung
belegungen = {}
matching_ergebnisse = []

for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    ta_monate = ta_ende - ta_start + 1
    aufwand_stunden = int(ta["aufwand"] * 160)  # PM → Stunden
    reststunden = aufwand_stunden
    kandidaten = []

    # Filtere Personen nach Kompetenz
    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        # Summe verfügbarer Stunden in Zeitraum
        verf_stunden = 0
        monatliche_reststunden = {}
        for monat in range(ta_start, ta_ende + 1):
            monat_label = f"{monat:02d}/2025"
            verf_monat_pm = person.get(f"verfuegbarkeit_{monat_label}", 0)
            verf_monat_h = verf_monat_pm * 160
            belegt_monat_h = belegungen.get(person["id"], {}).get(monat_label, 0)
            rest = max(0, verf_monat_h - belegt_monat_h)
            monatliche_reststunden[monat_label] = rest
            verf_stunden += rest

        if verf_stunden > 0:
            kandidaten.append((person, verf_stunden, monatliche_reststunden))

    # Sortierung: wer hat mehr Stunden verfügbar
    kandidaten = sorted(kandidaten, key=lambda x: x[1], reverse=True)

    # Zuweisung
    for person, verf_stunden, rest_by_month in kandidaten:
        if reststunden <= 0:
            break

        pid = person["id"]
        belegungen.setdefault(pid, {})
        anteil_stunden = min(reststunden, verf_stunden)

        # Gleichmäßig über Monate verteilen
        stunden_verplant = 0
        for monat_label in rest_by_month:
            monat_rest = rest_by_month[monat_label]
            if monat_rest <= 0:
                continue
            zuweisung = min(monat_rest, anteil_stunden - stunden_verplant)
            if zuweisung <= 0:
                continue
            belegungen[pid][monat_label] = belegungen[pid].get(monat_label, 0) + zuweisung
            stunden_verplant += zuweisung
            if stunden_verplant >= anteil_stunden:
                break

        matching_ergebnisse.append({
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "projekt_id": ta["projekt_id"],
            "kompetenz": ta["kompetenz"],
            "person_id": pid,
            "name": person["name"],
            "zugewiesene_stunden": stunden_verplant,
            "personentage": round(stunden_verplant / 8, 2)
        })

        reststunden -= stunden_verplant

    print(f"Verfügbare Kandidaten für: {ta['bezeichnung']} → {len(kandidaten)}")


Verfügbare Kandidaten für: Entwurf CAD-Baugruppe → 16
Verfügbare Kandidaten für: Simulation FEM-Struktur → 18
Verfügbare Kandidaten für: Signalverarbeitung Sensor → 12
Verfügbare Kandidaten für: Entwicklung Python-Modul → 10
Verfügbare Kandidaten für: Statistische Auswertung → 11
Verfügbare Kandidaten für: Optische Analyse → 14
Verfügbare Kandidaten für: Datenbankmodellierung → 15
Verfügbare Kandidaten für: Thermodynamische Bewertung → 12


Matching-Logik

In [6]:
def berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen):
    """
    Prüft die Verfügbarkeit einer Person für eine gegebene Zeitspanne.

    Args:
        person (pd.Series): Personendatensatz.
        ta_start (int): Startmonat (numerisch).
        ta_ende (int): Endmonat (numerisch).
        belegungen (dict): Bisherige Belegungen.

    Returns:
        tuple: (bool verfügbar, float verfügbare Gesamtkapazität, dict verfügbarkeit je Monat)
    """
    person_id = person["id"]
    verfügbar = True
    verfügbarkeit_summe = 0
    monatliche_restwerte = {}

    for monat in range(ta_start, ta_ende + 1):
        monat_label = f"{monat:02d}/2025"
        
        # Skip, wenn andere Projektbelegung vorliegt
        belegung_label = f"projektbelegung_{monat_label}"
        aktuelle_belegung = person.get(belegung_label, "Frei")
        if aktuelle_belegung not in ["Frei", ta["projekt_id"]]:
            rest = 0
        else:
            verfügbarkeit = person.get(f"verfuegbarkeit_{monat_label}", 0)
            belegt = belegungen.get(person_id, {}).get(monat_label, 0)
            rest = max(0, verfügbarkeit - belegt)

        monatliche_restwerte[monat_label] = rest

        if rest <= 0:
            verfügbar = False
            break
        verfügbarkeit_summe += rest

    return verfügbar, verfügbarkeit_summe, monatliche_restwerte


In [7]:
# Hauptloop über alle Teilaufgaben
for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    ta_monate = ta_ende - ta_start + 1
    ta_monatsaufwand = ta["aufwand"] / ta_monate
    restaufwand = ta["aufwand"]
    kandidaten = []

    # Filtere Personen, die die benötigte Kompetenz besitzen
    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        verfügbar, verf_summe, monatliche_restwerte = berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen)

        if verfügbar and verf_summe > 0:
            score = round(verf_summe * person["zeitbudget"], 2)
            kandidaten.append((person, score, verf_summe, monatliche_restwerte))

    # Sortiere Kandidaten nach ihrer nutzbaren Kapazität (absteigend)
    kandidaten = sorted(kandidaten, key=lambda x: x[1], reverse=True)

    # Weise Kapazität zu, bis Aufwand gedeckt ist oder keine Ressourcen mehr vorhanden
    for person, score, verf_summe, eintrag_monate in kandidaten:
        if restaufwand <= 0:
            break

        pid = person["id"]
        if pid not in belegungen:
            belegungen[pid] = {}

        verfügbarkeit_gesamt = sum(eintrag_monate.values())
        if verfügbarkeit_gesamt == 0:
            continue

        anteil = min(restaufwand, verfügbarkeit_gesamt)
        monatsanteil = anteil / ta_monate

        # Speichere Matching-Ergebnis
        matching_ergebnisse.append({
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "projekt_id": ta["projekt_id"],
            "kompetenz": ta["kompetenz"],
            "person_id": pid,
            "name": person["name"],
            "zugewiesener_aufwand": round(anteil, 2)
        })

        # Aktualisiere belegte Verfügbarkeiten
        for monat_label in eintrag_monate:
            verbrauch = min(monatsanteil, eintrag_monate[monat_label])
            belegungen[pid][monat_label] = belegungen[pid].get(monat_label, 0) + verbrauch

        restaufwand -= anteil

    print(f"Verfügbare Kandidaten für: {ta['bezeichnung']} → {len(kandidaten)}")


Verfügbare Kandidaten für: Entwurf CAD-Baugruppe → 0
Verfügbare Kandidaten für: Simulation FEM-Struktur → 0
Verfügbare Kandidaten für: Signalverarbeitung Sensor → 0
Verfügbare Kandidaten für: Entwicklung Python-Modul → 0
Verfügbare Kandidaten für: Statistische Auswertung → 0
Verfügbare Kandidaten für: Optische Analyse → 0
Verfügbare Kandidaten für: Datenbankmodellierung → 0
Verfügbare Kandidaten für: Thermodynamische Bewertung → 0


Ergebnisse speichern

In [8]:
df_matching = pd.DataFrame(matching_ergebnisse)
df_matching.to_csv("matching_ergebnis.csv", index=False)
print("Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'")

Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'


Neue Liste für Gantt-Daten

In [9]:
df_gantt = []

Vorbereitung Gantt-Diagramm

In [10]:
for eintrag in matching_ergebnisse:
    ta = teilaufgaben_df[teilaufgaben_df["teilaufgabe_id"] == eintrag["teilaufgabe_id"]].iloc[0]
    person = personen_df[personen_df["id"] == eintrag["person_id"]].iloc[0]

    person_id = person["id"]
    name = person["name"]
    aufgabe = ta["bezeichnung"]
    aufwand_stunden = eintrag["zugewiesene_stunden"]

    stunden_gesamt = int(aufwand_stunden * 160)
    reststunden = stunden_gesamt

    start_monat = int(ta["start"].split("/")[0])
    end_monat = int(ta["ende"].split("/")[0])

    start_date = datetime(2025, start_monat, 1)
    end_date = datetime(2025, end_monat, 28)

    aktuelles_datum = start_date
    block_start = None
    block_stunden = 0

    while aktuelles_datum <= end_date and reststunden > 0:
        monat_label = aktuelles_datum.strftime("%m/2025")
        verfuegbarkeit = person.get(f"verfuegbarkeit_{monat_label}", 0)
        bereits_belegt = belegungen.get(person_id, {}).get(monat_label, 0)

        if verfuegbarkeit - bereits_belegt <= 0:
            # kein Spielraum → neuen Block abschließen
            if block_start:
                block_ende = aktuelles_datum
                df_gantt.append({
                    "Aufgabe": aufgabe,
                    "Person": name,
                    "Start": block_start,
                    "Ende": block_ende,
                    "Stunden": block_stunden
                })
                block_start = None
                block_stunden = 0
            aktuelles_datum += timedelta(days=1)
            continue

        if block_start is None:
            block_start = aktuelles_datum

        # plane 1 Arbeitstag (8h)
        df_tag = aktuelles_datum
        belegungen.setdefault(person_id, {})
        belegungen[person_id][monat_label] = belegungen[person_id].get(monat_label, 0) + (8 / 160)
        block_stunden += 8
        reststunden -= 8
        aktuelles_datum += timedelta(days=1)

    # letztes offenes Zeitfenster speichern
    if block_start and block_stunden > 0:
        df_gantt.append({
            "Aufgabe": aufgabe,
            "Person": name,
            "Start": block_start,
            "Ende": aktuelles_datum,
            "Stunden": block_stunden
        })

Speichere Gantt-Daten

In [11]:
pd.DataFrame(df_gantt).to_csv("gantt_daten.csv", index=False)
print("Gantt-Daten gespeichert in 'gantt_daten.csv'")

Gantt-Daten gespeichert in 'gantt_daten.csv'


Berechnung des Effizienz-Scores

In [ ]:
def berechne_effizienz_score(zuweisungen_df):
    """Berechnet eine Effizienzmetrik basierend auf Anzahl Mitarbeiter und Pausenzeiten."""
    # Nur relevante Spalten verwenden: Mitarbeiter, Start, Ende
    df = zuweisungen_df.copy()
    df["start"] = pd.to_datetime(df["start"])
    df["ende"] = pd.to_datetime(df["ende"])

    # 1. Anzahl eingesetzter Mitarbeiter
    eingesetzte_mitarbeiter = df["mitarbeiter"].nunique()

    # 2. Pausen pro Mitarbeiter berechnen
    gesamtpause = timedelta(0)

    for mid, gruppe in df.groupby("mitarbeiter"):
        gruppe = gruppe.sort_values("start")
        for i in range(1, len(gruppe)):
            pause = gruppe.iloc[i]["start"] - gruppe.iloc[i - 1]["ende"]
            if pause > timedelta(0):
                gesamtpause += pause

    # Effizienz-Score (je kleiner desto besser)
    score = eingesetzte_mitarbeiter + gesamtpause.total_seconds() / 3600  # Pausen in Stunden

    print(f"Anzahl eingesetzter Mitarbeiter: {eingesetzte_mitarbeiter}")
    print(f"Gesamte Pausenzeit: {gesamtpause}")
    print(f"Effizienz-Score: {score:.2f}")
    return score

# Beispielaufruf (ggf. anpassen je nach Struktur der Zuweisungstabelle)
# ... berechne_effizienz_score(zuweisungen_df)
